--------------------------------------------------------------------------------

# Created by [Nikolaos G. Giakoumoglou](https://github.com/giakou4)

### Electrical and Computer Engineer

Mobile: +30 6906430995 | e-mail: [ngiakoumoglou@gmail.gr](mailto:ngiakoumoglou@gmail.gr) | e-mail: [ngiakoumoglou@hotmail.gr](mailto:ngiakoumoglou@hotmail.gr) |

-------------------------------------------------------------------------------

# YOLOv3

This tutorial is based on the [YOLOv3 repository](https://github.com/ultralytics/yolov3) by [Ultralytics](https://www.ultralytics.com/). 

# 1 Install Dependencies

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
%pip install -qr requirements.txt 

!pip install pycocotools
!pip install onnx>=1.9.0
!pip install roboflow

import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# 2 Download Dataset 

We'll download our dataset from [Roboflow](https://roboflow.com/). Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.


In [ ]:
%%capture
from roboflow import Roboflow
rf = Roboflow(api_key="7EOGxE0Zhj6CaawbEz1J")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("yolov5")

## 2.1 Create custom yaml files for each model

In [ ]:
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
!cp -r ./models/yolov3.yaml ./models/custom_yolov3.yaml
%cd models
!sed -i 's/nc: 80/nc: {num_classes}/' custom_yolov3.yaml
%cd ..

# 3 Train Custom YOLOv3 Detector

To kick off training we running the training command with the following options:

* _img_: define input image size
* _batch_: determine batch size
* _epochs_: define the number of training epochs
* _data_: set the path to our yaml file
* _cfg_: specify our model configuration
* _weights_: specify a custom path to weights
* _name_: result names
* _nosave_: only save the final checkpoint
* _cache_: cache images for faster training
* _hyps_: hyperparameters of training

In [ ]:
img =   1024 #@param {type:"integer"}
batch =  4 #@param {type:"integer"}
epochs = 300 #@param {type:"integer"}
cfg = 'models/custom_yolov3.yaml' #@param ["models/custom_yolov3.yaml"]
weights = 'yolov3.pt' #@param ["yolov3.pt"]
hyp = 'data/hyps/hyp.scratch-low.yaml' #@param["data/hyps/hyp.scratch-low.yaml", "data/hyps/hyp.scratch-med.yaml", "data/hyps/hyp.scratch-high.yaml"]
name = 'experiment-1' #@param {type:"string"}

In [ ]:
%%time
%cd /content/yolov3/
!python train.py --img {img} --batch {batch} --epochs {epochs} --data {dataset.location}/data.yaml --cfg {cfg} --weights {weights} --adam --hyp {hyp} --name {name} --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 4 Validate Custom YOLOv3 Detector 



In [ ]:
!python val.py --weights runs/train/{name}}/weights/best.pt --img {img} --data {dataset.location}/data.yaml --name {name} --task val